# Exploring Legacy Chains in LangChain

In [1]:
!pip install langchain==0.2.0
!pip install langchain-openai==0.1.7
!pip install langchain-community==0.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.1/397.1 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.2.3
    Uninstalling langsmith-0.2.3:
      Successfully uninstalled langsmith-0.2.3
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.3
    Uninstalling langchain-text-splitters-0.3.3:
      

In [2]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPEN_API_KEY')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HF_TOKEN')

## Load Connection to LLM

Here we create a connection to ChatGPT to use later in our chains

In [4]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

In [5]:
reviews = [
    f"""
    Purchased this adorable koala plush toy for my nephew's birthday,
    and he's absolutely smitten with it, carrying it around everywhere he goes.
    The plush is incredibly soft, and the koala's face has an endearing expression.
    However, I did find it a tad on the smaller side given its price point.
    I believe there may be larger alternatives available at a similar price.
    To my delight, it arrived a day earlier than anticipated,
    allowing me to enjoy it briefly before gifting it to him.
    """,
    f"""
    Required a stylish lamp for my office space, and this particular one
    came with added storage at a reasonable price.
    The delivery was surprisingly quick, arriving within just two days.
    However, the pull string for the lamp suffered damage during transit.
    To my relief, the company promptly dispatched a replacement,
    which arrived within a few days. Assembly was a breeze.
    Then, I encountered an issue with a missing component,
    but their support team responded swiftly and provided the missing part.
    It appears to be a commendable company that genuinely values its
    customers and the quality of its products.
    """
    ]

In [6]:
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate

prompt = """
            Act as a product review analyst.
            Your task is to generate a short summary of a product
            review from an ecommerce site.

            Generate a summary of the review (max 2 lines)
            Also show both the positives and negatives from the review (max 2 bullets)

            ```{review}```
"""

prompt_template = ChatPromptTemplate.from_template(prompt)
llm_chain = LLMChain(llm=chatgpt, prompt=prompt_template)

<ipython-input-6-ad87462f2ba5>:16: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  llm_chain = LLMChain(llm=chatgpt, prompt=prompt_template)


In [7]:
reviews[0]

"\n    Purchased this adorable koala plush toy for my nephew's birthday,\n    and he's absolutely smitten with it, carrying it around everywhere he goes.\n    The plush is incredibly soft, and the koala's face has an endearing expression.\n    However, I did find it a tad on the smaller side given its price point.\n    I believe there may be larger alternatives available at a similar price.\n    To my delight, it arrived a day earlier than anticipated,\n    allowing me to enjoy it briefly before gifting it to him.\n    "

In [8]:
result = llm_chain.invoke(  {'review': reviews[0]}  )
result

{'review': "\n    Purchased this adorable koala plush toy for my nephew's birthday,\n    and he's absolutely smitten with it, carrying it around everywhere he goes.\n    The plush is incredibly soft, and the koala's face has an endearing expression.\n    However, I did find it a tad on the smaller side given its price point.\n    I believe there may be larger alternatives available at a similar price.\n    To my delight, it arrived a day earlier than anticipated,\n    allowing me to enjoy it briefly before gifting it to him.\n    ",
 'text': "Summary: Adorable koala plush toy loved by recipient, but slightly small for the price.\n\nPositives:\n- Incredibly soft plush material\n- Endearing expression on the koala's face\n\nNegatives:\n- Smaller size than expected for the price\n- Possibly larger alternatives available at a similar price"}

In [9]:
print(result['text'])

Summary: Adorable koala plush toy loved by recipient, but slightly small for the price.

Positives:
- Incredibly soft plush material
- Endearing expression on the koala's face

Negatives:
- Smaller size than expected for the price
- Possibly larger alternatives available at a similar price


In [10]:
prompt_template

ChatPromptTemplate(input_variables=['review'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['review'], template='\n            Act as a product review analyst.\n            Your task is to generate a short summary of a product\n            review from an ecommerce site.\n\n            Generate a summary of the review (max 2 lines)\n            Also show both the positives and negatives from the review (max 2 bullets)\n\n            ```{review}```\n'))])

In [11]:
formatted_reviews = [{'review': review}
                        for review in reviews]

results = llm_chain.map().invoke(formatted_reviews)
len(results)

2

In [12]:
for result in results:
    print(result['text'])
    print()

Summary: Adorable koala plush toy loved by recipient, but slightly small for the price.

Positives:
- Incredibly soft plush material
- Endearing expression on the koala's face

Negatives:
- Smaller size than expected for the price
- Larger alternatives available at a similar price

Summary: Stylish lamp with added storage, quick delivery, excellent customer service.

Positives:
- Stylish lamp with added storage
- Quick delivery and excellent customer service

Negatives:
- Pull string for lamp suffered damage during transit
- Encountered an issue with a missing component



### Combining Multiple Tasks with SequentialChain

The next step after calling a language model is to make a series of calls to a language model. This is particularly useful when you want to take the output from one call and use it as the input to another.

Sequential chains allow you to connect multiple chains and compose them into pipelines that execute some specific scenario.

Here given a few IT Support issues, for each customer issue:
- We want to detect the customer message language
- We want to translate the customer message from the source language to English
- We want the AI to generate a suitable response to the problem in English
- We want to translate this response to the customer language

In [14]:
it_support_queue = [
    "I can't access my email. It keeps showing an error message. Please help.",
    "Tengo problemas con la VPN. No puedo conectarme a la red de la empresa. ¿Pueden ayudarme, por favor?",
    "Mon imprimante ne répond pas et n'imprime plus. J'ai besoin d'aide pour la réparer.",
    "我无法访问公司的网站。每次都显示错误信息。请帮忙解决。"
]

it_support_queue

["I can't access my email. It keeps showing an error message. Please help.",
 'Tengo problemas con la VPN. No puedo conectarme a la red de la empresa. ¿Pueden ayudarme, por favor?',
 "Mon imprimante ne répond pas et n'imprime plus. J'ai besoin d'aide pour la réparer.",
 '我无法访问公司的网站。每次都显示错误信息。请帮忙解决。']

In [15]:
  # Chain 1: Detect customer message language
prompt1 = """
  Act as a customer support agent.
  For the customer support message delimited below by triple backticks,
  Output the language of the message in one word only, e.g. Spanish

  Customer Message:
  ```{orig_msg}```
"""

prompt_template1 = ChatPromptTemplate.from_template(prompt1)
chain1 = LLMChain(llm=chatgpt, prompt=prompt_template1, output_key="orig_lang")


In [16]:
# Chain 2: Translate Customer Message to English
prompt2 = """
  Act as a customer support agent.
  For the customer message and customer message language delimited below by triple backticks,
  Translate the customer message from the customer message language to English
  if customer message language is not in English,
  else return back the original customer message.

  Customer Message:
  ```{orig_msg}```
  Customer Message Language:
  ```{orig_lang}```
"""
prompt_template2 = ChatPromptTemplate.from_template(prompt2)
chain2 = LLMChain(llm=chatgpt, prompt=prompt_template2, output_key="trans_msg")

In [17]:
# Chain 3: Generate a resolution response in English
prompt3 = """
  Act as a customer support agent.
  For the customer support message delimited below by triple backticks,
  Generate an appropriate resolution response in English.

  Customer Message:
  ```{trans_msg}```
"""
prompt_template3 = ChatPromptTemplate.from_template(prompt3)
chain3 = LLMChain(llm=chatgpt, prompt=prompt_template3, output_key="trans_response")

In [18]:
# Chain 4: Translate resolution response from English to Customer's language
prompt4 = """
  Act as a customer support agent.
  For the customer resolution response and target language delimited below by triple backticks,
  Translate the customer resolution response message from English to the target language
  if target language is not in English,
  else return back the original customer resolution response.

  Customer Resolution Response:
  ```{trans_response}```
  Target Language:
  ```{orig_lang}```
"""
prompt_template4 = ChatPromptTemplate.from_template(prompt4)
chain4 = LLMChain(llm=chatgpt, prompt=prompt_template4, output_key="response")

In [19]:
from langchain.chains import SequentialChain

# combine all chains sequentially
seq_chain = SequentialChain(
    chains=[chain1, chain2, chain3, chain4],
    input_variables=["orig_msg"], # input data
    output_variables=["orig_msg", "orig_lang","trans_msg", "response", "trans_response"], # response data
    verbose=True # set to False to turn of debugging messages
)

In [20]:
support_msgs_formatted = [{'orig_msg': msg} for msg in it_support_queue]
support_msgs_formatted

[{'orig_msg': "I can't access my email. It keeps showing an error message. Please help."},
 {'orig_msg': 'Tengo problemas con la VPN. No puedo conectarme a la red de la empresa. ¿Pueden ayudarme, por favor?'},
 {'orig_msg': "Mon imprimante ne répond pas et n'imprime plus. J'ai besoin d'aide pour la réparer."},
 {'orig_msg': '我无法访问公司的网站。每次都显示错误信息。请帮忙解决。'}]

In [21]:
# run the chain on all the support messages
results = seq_chain.map().invoke(support_msgs_formatted)



> Entering new SequentialChain chain...


> Entering new SequentialChain chain...


> Entering new SequentialChain chain...


> Entering new SequentialChain chain...

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.


In [22]:
results[2]

{'orig_msg': "Mon imprimante ne répond pas et n'imprime plus. J'ai besoin d'aide pour la réparer.",
 'orig_lang': 'French',
 'trans_msg': 'My printer is not responding and not printing anymore. I need help to fix it.',
 'response': "Résolution:\nJe suis désolé d'apprendre que vous rencontrez des problèmes avec votre imprimante. Pour résoudre ce problème, veuillez essayer les étapes suivantes:\n1. Vérifiez si l'imprimante est correctement connectée à la source d'alimentation et allumée.\n2. Assurez-vous qu'il y a suffisamment de papier dans le bac de l'imprimante et qu'il n'y a pas de bourrage papier.\n3. Redémarrez l'imprimante et votre ordinateur.\n4. Mettez à jour les pilotes de l'imprimante sur votre ordinateur.\nSi le problème persiste après avoir essayé ces étapes, veuillez nous le faire savoir afin que nous puissions vous aider davantage.",
 'trans_response': "Resolution:\nI'm sorry to hear that you're experiencing issues with your printer. To troubleshoot this problem, please tr

In [23]:
import pandas as pd

df = pd.DataFrame(results)
df

,orig_msg,orig_lang,trans_msg,response,trans_response
0,I can't access my email. It keeps showing an e...,English,I can't access my email. It keeps showing an e...,Resolution Response:\nI apologize for the inco...,Resolution Response:\nI apologize for the inco...
1,Tengo problemas con la VPN. No puedo conectarm...,Spanish,I'm having issues with the VPN. I can't connec...,Resolución:\nLamento escuchar que estás tenien...,Resolution:\nI'm sorry to hear that you're hav...
2,Mon imprimante ne répond pas et n'imprime plus...,French,My printer is not responding and not printing ...,Résolution:\nJe suis désolé d'apprendre que vo...,Resolution:\nI'm sorry to hear that you're exp...
3,我无法访问公司的网站。每次都显示错误信息。请帮忙解决。,Chinese,I cannot access the company's website. An erro...,```Resolution:\n对于您正在经历的不便，我感到抱歉。为了更好地帮助您，您能否提...,Resolution:\nI apologize for the inconvenience...
